In [1]:
import pandas as pd
import json

In [2]:
with open('/Users/TimothyW/Fun/cricket_prediction/data/odis_male_json/64857.json', 'r') as f:
    d = json.load(f)


In [3]:
## Aim to flatten everything so each delivery is a flat dict
# start with a single delivery

# ultimate aim is df with  wickets | runs | balls left | target_runs

In [51]:
def process_delivery(jsn):
    if 'wickets' in jsn.keys():
        wicket_player_out = jsn['wickets'][0]['player_out']
        wicket_kind = jsn['wickets'][0]['kind']
        wicket_taken = True
    else:
        wicket_player_out = None
        wicket_kind = None
        wicket_taken = False
    return {
        "batter": jsn["batter"],
        "bowler": jsn["bowler"],
        "non_striker": jsn["non_striker"],
        "batter_runs": jsn["runs"]["batter"],
        "extras_runs": jsn["runs"]["extras"],
        "total_delivery_runs": jsn["runs"]["total"],
        "wicket_player_out": wicket_player_out,
        "wicket_kind": wicket_kind,
        "wicket_taken": wicket_taken
    }


In [53]:
process_delivery(d['innings'][0]['overs'][9]['deliveries'][1])

{'batter': 'Saleem Elahi',
 'bowler': 'JDP Oram',
 'non_striker': 'Imran Farhat',
 'batter_runs': 0,
 'extras_runs': 0,
 'total_delivery_runs': 0,
 'wicket_player_out': None,
 'wicket_kind': None,
 'wicket_taken': False}

In [48]:
d['innings'][0]['overs'][9]

{'over': 9,
 'deliveries': [{'batter': 'Yasir Hameed',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0},
   'wickets': [{'player_out': 'Yasir Hameed', 'kind': 'bowled'}]},
  {'batter': 'Saleem Elahi',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'Saleem Elahi',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'Saleem Elahi',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'Saleem Elahi',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'Saleem Elahi',
   'bowler': 'JDP Oram',
   'non_striker': 'Imran Farhat',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}}]}

In [152]:
def process_over(jsn):
    deliv_list = []
    ref_dict = {'over_n':jsn['over']}
    delivery_list = jsn['deliveries']
    fraction_ball = 1./len(delivery_list)
    for ball_i in range(0, len(delivery_list)):
        delivery = process_delivery(delivery_list[ball_i])
        delivery['over_ball_n'] = ball_i + 1
        delivery['over_fractional'] = ball_i * fraction_ball
        delivery['overall_ball_fraction'] = jsn['over'] + delivery['over_fractional']
        delivery_ref = dict(ref_dict, **delivery)
        deliv_list.append(delivery_ref)

    

    return deliv_list

        



In [153]:
process_over(d['innings'][0]['overs'][9])

[{'over_n': 9,
  'batter': 'Yasir Hameed',
  'bowler': 'JDP Oram',
  'non_striker': 'Imran Farhat',
  'batter_runs': 0,
  'extras_runs': 0,
  'total_delivery_runs': 0,
  'wicket_player_out': 'Yasir Hameed',
  'wicket_kind': 'bowled',
  'wicket_taken': True,
  'over_ball_n': 1,
  'over_fractional': 0.0,
  'overall_ball_fraction': 9.0},
 {'over_n': 9,
  'batter': 'Saleem Elahi',
  'bowler': 'JDP Oram',
  'non_striker': 'Imran Farhat',
  'batter_runs': 0,
  'extras_runs': 0,
  'total_delivery_runs': 0,
  'wicket_player_out': None,
  'wicket_kind': None,
  'wicket_taken': False,
  'over_ball_n': 2,
  'over_fractional': 0.16666666666666666,
  'overall_ball_fraction': 9.166666666666666},
 {'over_n': 9,
  'batter': 'Saleem Elahi',
  'bowler': 'JDP Oram',
  'non_striker': 'Imran Farhat',
  'batter_runs': 0,
  'extras_runs': 0,
  'total_delivery_runs': 0,
  'wicket_player_out': None,
  'wicket_kind': None,
  'wicket_taken': False,
  'over_ball_n': 3,
  'over_fractional': 0.3333333333333333,
  '

In [241]:
def process_innings(jsn):    
    l = []
    for over in jsn['overs']:
        pr_over = process_over(over)        
        l += pr_over
        
        
    innings_df = pd.DataFrame(l)
    innings_df['batting_team'] = jsn['team']
    innings_df['first_innings_total_runs'] = innings_df['total_delivery_runs'].sum()
    innings_df['current_runs'] = innings_df['total_delivery_runs'].cumsum()
    innings_df['current_wickets'] = innings_df['wicket_taken'].cumsum()
    innings_df['overall_ball_n'] = np.arange( len(innings_df))
    innings_df['total_balls_bowled'] = len(innings_df)
    innings_df['innings_balls_left'] = len(innings_df) - innings_df['overall_ball_n']
    innings_df['total_wickets_lost'] = innings_df['wicket_taken'].sum()
    
    return innings_df

In [242]:
innings_df = process_innings(d['innings'][0])

In [243]:
innings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   over_n                    304 non-null    int64  
 1   batter                    304 non-null    object 
 2   bowler                    304 non-null    object 
 3   non_striker               304 non-null    object 
 4   batter_runs               304 non-null    int64  
 5   extras_runs               304 non-null    int64  
 6   total_delivery_runs       304 non-null    int64  
 7   wicket_player_out         7 non-null      object 
 8   wicket_kind               7 non-null      object 
 9   wicket_taken              304 non-null    bool   
 10  over_ball_n               304 non-null    int64  
 11  over_fractional           304 non-null    float64
 12  overall_ball_fraction     304 non-null    float64
 13  batting_team              304 non-null    object 
 14  first_inni

In [234]:
innings_df['first_innings_total_runs'] = innings_df['total_delivery_runs'].sum()

In [200]:
innings_df['current_runs'] = innings_df['total_delivery_runs'].cumsum()

In [201]:
innings_df['current_wickets'] = innings_df['wicket_taken'].cumsum()

In [202]:
innings_df = innings_df.sort_values('overall_ball_fraction', ascending=True)

In [203]:
import numpy as np

In [222]:
def process_match(jsn):
    first_innings_df = process_innings(jsn['innings'][0])
    first_innings_df['data_version'] = jsn['meta']['data_version']
    first_innings_df['date_created'] = jsn['meta']['created']
    first_innings_df['revision'] = jsn['meta']['revision']
    first_innings_df['city'] = jsn['info'].get('city', None)
    first_innings_df['date'] = jsn['info']['dates'][0]
    first_innings_df['match_type'] = jsn['info']['match_type']
    first_innings_df['toss_winner'] = jsn['info']['toss']['winner']
    first_innings_df['team_one'] = jsn['info']['teams'][0]
    first_innings_df['team_two'] = jsn['info']['teams'][1]
    return first_innings_df

In [223]:
process_match(d)

,over_n,batter,bowler,non_striker,batter_runs,extras_runs,total_delivery_runs,wicket_player_out,wicket_kind,wicket_taken,...,innings_balls_left,data_version,date_created,revision,city,date,match_type,toss_winner,team_one,team_two
0,0,Yasir Hameed,DR Tuffey,Imran Farhat,4,0,4,None,None,False,...,304,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
1,0,Yasir Hameed,DR Tuffey,Imran Farhat,0,0,0,None,None,False,...,303,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
2,0,Yasir Hameed,DR Tuffey,Imran Farhat,0,0,0,None,None,False,...,302,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
3,0,Yasir Hameed,DR Tuffey,Imran Farhat,0,0,0,None,None,False,...,301,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
4,0,Yasir Hameed,DR Tuffey,Imran Farhat,0,0,0,None,None,False,...,300,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,49,Azhar Mahmood,DR Tuffey,Moin Khan,4,0,4,None,None,False,...,5,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
300,49,Azhar Mahmood,DR Tuffey,Moin Khan,0,0,0,None,None,False,...,4,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
301,49,Azhar Mahmood,DR Tuffey,Moin Khan,1,0,1,None,None,False,...,3,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand
302,49,Moin Khan,DR Tuffey,Azhar Mahmood,4,0,4,None,None,False,...,2,1.0.0,2021-03-08,1,Auckland,2004-01-03,ODI,New Zealand,Pakistan,New Zealand


In [224]:
import os

In [235]:
def process_file(f_name):
    with open(f_name, 'r') as f:
        d = json.load(f)
    return process_match(d)

In [244]:
data_path = '/Users/TimothyW/Fun/cricket_prediction/data/odis_male_json/'
dfs = []
for f_name in os.listdir(data_path):
    f_name_full = f"{data_path}/{f_name}"    
    dfs.append(process_file(f_name_full))

In [245]:
df_all = pd.concat(dfs)

In [246]:
df_all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675146 entries, 0 to 299
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   over_n                    675146 non-null  int64  
 1   batter                    675146 non-null  object 
 2   bowler                    675146 non-null  object 
 3   non_striker               675146 non-null  object 
 4   batter_runs               675146 non-null  int64  
 5   extras_runs               675146 non-null  int64  
 6   total_delivery_runs       675146 non-null  int64  
 7   wicket_player_out         18416 non-null   object 
 8   wicket_kind               18416 non-null   object 
 9   wicket_taken              675146 non-null  bool   
 10  over_ball_n               675146 non-null  int64  
 11  over_fractional           675146 non-null  float64
 12  overall_ball_fraction     675146 non-null  float64
 13  batting_team              675146 non-null  obje

In [239]:
df_all.to_parquet('/Users/TimothyW/Fun/cricket_prediction/data/processed_first_innings/first_innings_processed.parquet')

,over_n,batter,bowler,non_striker,batter_runs,extras_runs,total_delivery_runs,wicket_player_out,wicket_kind,wicket_taken,...,total_wickets_lost,data_version,date_created,revision,city,date,match_type,toss_winner,team_one,team_two
0,0,Vikramjit Singh,AS Joseph,MP O'Dowd,0,0,0,None,None,False,...,7,1.1.0,2022-05-31,1,Amstelveen,2022-05-31,ODI,West Indies,Netherlands,West Indies
96,15,Musa Ahmed,AJ Hosein,MP O'Dowd,1,0,1,None,None,False,...,7,1.1.0,2022-05-31,1,Amstelveen,2022-05-31,ODI,West Indies,Netherlands,West Indies
90,14,MP O'Dowd,KR Mayers,Musa Ahmed,1,0,1,None,None,False,...,7,1.1.0,2022-05-31,1,Amstelveen,2022-05-31,ODI,West Indies,Netherlands,West Indies
91,14,Musa Ahmed,KR Mayers,MP O'Dowd,0,0,0,None,None,False,...,7,1.1.0,2022-05-31,1,Amstelveen,2022-05-31,ODI,West Indies,Netherlands,West Indies
92,15,MP O'Dowd,AJ Hosein,Musa Ahmed,1,0,1,None,None,False,...,7,1.1.0,2022-05-31,1,Amstelveen,2022-05-31,ODI,West Indies,Netherlands,West Indies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,21,RR Sarwan,M Ntini,RL Powell,0,0,0,None,None,False,...,8,1.0.0,2021-03-08,1,Durban,2004-01-30,ODI,West Indies,West Indies,South Africa
128,20,RL Powell,RJ Peterson,RR Sarwan,0,0,0,None,None,False,...,8,1.0.0,2021-03-08,1,Durban,2004-01-30,ODI,West Indies,West Indies,South Africa
127,20,RL Powell,RJ Peterson,RR Sarwan,0,0,0,None,None,False,...,8,1.0.0,2021-03-08,1,Durban,2004-01-30,ODI,West Indies,West Indies,South Africa
126,20,RL Powell,RJ Peterson,RR Sarwan,2,0,2,None,None,False,...,8,1.0.0,2021-03-08,1,Durban,2004-01-30,ODI,West Indies,West Indies,South Africa
